In [21]:
# this is to create a ccpd training dataset
# the target is simple, 
# 1. extract the place area/ or the original from the image
# 2. use the province as it's label
# 3. save an annotation label file

import os
import sys
import cv2
import numpy as np
import random
import shutil
import json
import matplotlib
import matplotlib.pyplot as plt
# import pillow
from PIL import Image

ccpd_root = '../data/CCPD2019/'
# show the image
ccpd_base = ccpd_root + 'ccpd_base/'
images = os.listdir(ccpd_base)
# first_image = images[6]
# first_anno = first_image.split('-')
# image = Image.open(ccpd_base + first_image)
# plt.imshow(image)
# plt.show()

# first, calculate the number of images which ratio is > 0.5
count = 0
for image in images:
    ratio = image.split('-')[0]
    l_ratio = len(ratio)
    f_ratio = float(ratio)/10**l_ratio
    if f_ratio > 0.03:
        count += 1
print('images with ratio > 0.03: ', count)

images with ratio > 0.5:  43870


In [33]:
# store the image with ratio > 0.03
train_img_label = {}
test_img_label = {}
val_img_label = {}

count = 0
random.seed(2024)  
ccpd_large = '../data/CCPD_large/'
ccpd_small = '../data/CCPD_small/'
from tqdm import tqdm
for image in tqdm(images):
    image_split = image.split('-')
    ratio = image_split[0]
    l_ratio = len(ratio)
    f_ratio = float(ratio)/10**l_ratio
    if f_ratio < 0.03:
        continue
    bbox = image_split[2]
    bbox = bbox.split('_')
    bbox_p1 = bbox[0].split('&')
    bbox_p2 = bbox[1].split('&')

    # new count name 
    new_image_name = str(count) + '.jpg'
    # label = image_split[4]
    # label = label.split('_')[0]
    # label = int(label)
    label = image_split[4]
    label = label.split('_')[-1]
    label = int(label)
    # get a random value, if 0-0.7 train, 0.7-0.8 val, 0.8-1 test
    rand = random.random()
    if rand < 0.7:
        new_dir = 'train/'
        if not os.path.exists(ccpd_large + new_dir):
            os.makedirs(ccpd_large + new_dir)
        if not os.path.exists(ccpd_small + new_dir):
            os.makedirs(ccpd_small + new_dir)
        train_img_label[new_image_name] = label
    elif rand < 0.8:
        new_dir = 'val/'
        if not os.path.exists(ccpd_large + new_dir):
            os.makedirs(ccpd_large + new_dir)
        if not os.path.exists(ccpd_small + new_dir):
            os.makedirs(ccpd_small + new_dir)
        val_img_label[new_image_name] = label
    else:
        new_dir = 'test/'
        if not os.path.exists(ccpd_large + new_dir):
            os.makedirs(ccpd_large + new_dir)
        if not os.path.exists(ccpd_small + new_dir):
            os.makedirs(ccpd_small + new_dir)
        test_img_label[new_image_name] = label
    # store the image to the new dir in bmp
    img = Image.open(ccpd_base + image)
    img.save(ccpd_large + new_dir + new_image_name)

    count += 1

    # crop the image
    img = img.crop((int(bbox_p1[0]), int(bbox_p1[1]), int(bbox_p2[0]), int(bbox_p2[1])))
    img.save(ccpd_small + new_dir + new_image_name)

# store the label
with open(ccpd_large + 'train_label.json', 'w') as f:
    json.dump(train_img_label, f)
with open(ccpd_large + 'val_label.json', 'w') as f:
    json.dump(val_img_label, f)
with open(ccpd_large + 'test_label.json', 'w') as f:
    json.dump(test_img_label, f)

# store the label
with open(ccpd_small + 'train_label.json', 'w') as f:
    json.dump(train_img_label, f)
with open(ccpd_small + 'val_label.json', 'w') as f:
    json.dump(val_img_label, f)
with open(ccpd_small + 'test_label.json', 'w') as f:
    json.dump(test_img_label, f)

print('done')
    


    


100%|██████████| 199996/199996 [02:44<00:00, 1217.18it/s]

done


In [16]:
# this is to create a ccpd training dataset
# the target is simple, 
# 1. extract the place area/ or the original from the image
# 2. use the province as it's label
# 3. save an annotation label file

import os
import sys
import cv2
import numpy as np
import random
import shutil
import json
import matplotlib
import matplotlib.pyplot as plt
# import pillow
from PIL import Image

ccpd_root = '../data/CCPD2019/'
# show the image
ccpd_base = ccpd_root + 'ccpd_base/'
images = os.listdir(ccpd_base)
# first_image = images[6]
# first_anno = first_image.split('-')
# image = Image.open(ccpd_base + first_image)
# plt.imshow(image)
# plt.show()

# first, calculate the number of images which ratio is > 0.5
count = 0
import pandas as pd
table = pd.DataFrame(columns=['name', 'ratio', 'size', 'plate1',
                               'plate2', 'plate3', 'plate4', 'plate5',
                               'plate6', 'plate7'])
from tqdm import tqdm

name = []
ratios = []
sizes = []
plate1 = []
plate2 = []
plate3 = []
plate4 = []
plate5 = []
plate6 = []
plate7 = []

for image in tqdm(images):
    ratio, tilt, bbox, vertices, plate, bright, blur = image.split('-')
    l_ratio = len(ratio)
    f_ratio = float(ratio)/10**l_ratio

    plate_split = plate.split('_')

    bbox = bbox.split('_')
    bbox_p1 = bbox[0].split('&')
    bbox_p2 = bbox[1].split('&')
    bbox_size = (int(bbox_p2[0]) - int(bbox_p1[0])) * (int(bbox_p2[1]) - int(bbox_p1[1]))

    # add it to the pandas dataset
    name.append(image)
    ratios.append(f_ratio)
    sizes.append(bbox_size)
    plate1.append(plate_split[0])
    plate2.append(plate_split[1])
    plate3.append(plate_split[2])
    plate4.append(plate_split[3])
    plate5.append(plate_split[4])
    plate6.append(plate_split[5])
    plate7.append(plate_split[6])
    
table['name'] = name
table['ratio'] = ratios
table['size'] = sizes
table['plate1'] = plate1
table['plate2'] = plate2
table['plate3'] = plate3
table['plate4'] = plate4
table['plate5'] = plate5
table['plate6'] = plate6
table['plate7'] = plate7

table

100%|██████████| 199996/199996 [00:00<00:00, 566584.75it/s]


,name,ratio,size,plate1,plate2,plate3,plate4,plate5,plate6,plate7
0,016091954023-91_84-178&390_389&489-397&480_182...,0.016092,20889,0,0,31,24,33,21,24
1,0402274904214-90_91-156&490_513&621-529&613_16...,0.040227,46767,0,0,25,10,33,32,29
2,0110632183908-91_91-308&458_482&518-486&529_31...,0.011063,10440,0,0,29,33,23,26,33
3,0331561302682-92_87-129&406_450&515-457&513_14...,0.033156,34989,0,0,13,31,27,26,26
4,0299329501916-93_90-272&481_524&603-524&608_25...,0.029933,30744,0,0,19,26,24,28,28
...,...,...,...,...,...,...,...,...,...,...
199991,0300095785441-85_95-310&499_573&602-561&583_30...,0.030010,27089,0,0,33,26,28,8,29
199992,0165948275862-91_78-226&503_462&580-471&584_23...,0.016595,18172,0,0,6,30,24,28,31
199993,0405747126436-89_86-122&402_495&519-503&519_12...,0.040575,43641,0,0,5,14,30,24,25
199994,0290804597702-93_73-307&526_593&627-592&628_32...,0.029080,28886,0,0,0,32,11,33,24


In [35]:
large_img = table[table['ratio'] > 0.03]
print('images with ratio > 0.02: ', large_img.shape[0])
# count each last digit
large_img['plate7'].value_counts()
len(large_img['plate7'].unique())

images with ratio > 0.02:  43870


34